Ce notebook vise à utiliser les données journalières de pluie pour determiner si un jour est inclus dans la saison des pluies ou pas selon Dr. SALACK et Al.
Au finalement on aura un  dataframe sauvegardé dans le repertoire 

In [1]:
import pandas as pd
from config import Config

In [2]:
# Lecture du fichier de données de pluie et d'evapotranspiration prétraité
#df_jour = pd.read_csv('../DATA/merged_data.csv')
df_jour=pd.read_csv(str(Config.FILES_TRAITED_PATH) + '/' + str(Config.PLUIE_ETP_FILE_NAME))
df_jour.dropna(inplace=True)
df_jour['Date'] = pd.to_datetime(df_jour['Date'])

In [3]:
def get_decade(day):
    if day <= 10:
        return 1
    elif day <= 20:
        return 2
    else:
        return 3
def get_trimestre(month):
    if month <= 3:
        return 1
    elif month<= 6:
        return 2
    elif month<= 9:
        return 3
    else:
        return 4
def get_semestre(month):
    if month <= 6:
        return 1
    else:
        return 2
df_jour['Year'] = df_jour['Date'].dt.year
df_jour['Month'] = df_jour['Date'].dt.month
df_jour['Day'] = df_jour['Date'].dt.day
df_jour['Decade'] = df_jour['Day'].apply(get_decade)
df_jour['Trimestre']=df_jour['Month'].apply(get_trimestre)
df_jour['Semestre']=df_jour['Month'].apply(get_semestre)

df_jour['Saison_Pluie'] = False

grouped_data = df_jour.groupby(['Station', 'Year'])

for group_key, group_df in grouped_data:
    station = group_key[0]
    annee = group_key[1]
    
    premier_jour_pluie = group_df.loc[group_df['Pluie'] >= 9.75, 'Date'].min()
    dernier_jour_pluie = group_df.loc[group_df['Pluie'] > 0, 'Date'].max()
    
    if premier_jour_pluie is not pd.NaT and dernier_jour_pluie is not pd.NaT:
        debut_saison = premier_jour_pluie
        fin_saison = dernier_jour_pluie
        
        jours_saison_pluie = group_df.loc[(group_df['Date'] >= debut_saison) & (group_df['Date'] <= fin_saison), 'Date']
        df_jour.loc[jours_saison_pluie.index, 'Saison_Pluie'] = True

saison_df = df_jour[['Station', 'Year', 'Trimestre','Semestre','Month','Decade','Day', 'Saison_Pluie']]
saison_df.sort_values(by=['Station', 'Year', 'Month','Decade','Day'], ascending=True, inplace=True)

/var/folders/pk/rg_m87h56ws5wzxy5dg2rcmw0000gn/T/ipykernel_1838/1841180108.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  saison_df.sort_values(by=['Station', 'Year', 'Month','Decade','Day'], ascending=True, inplace=True)


In [4]:
#Sauvegarde d'un fichier test pour Boromo
saison_df[(saison_df['Station']=='Boromo')& (saison_df['Year']==2011)].to_csv(str(Config.FILES_TRAITED_PATH) + '/' + str(Config.STATION_TEST),index=False)

In [5]:
#Sauvegarde d'un fichier pour toutes les stations
saison_df.to_csv(str(Config.FILES_TRAITED_PATH) + '/' + str(Config.STATION_ALL),index=False)

In [6]:
saison_df.head()

,Station,Year,Trimestre,Semestre,Month,Decade,Day,Saison_Pluie
603,Bobo_Dioulasso,1961,1,1,1,1,1,False
11874,Bobo_Dioulasso,1961,1,1,1,1,2,False
23145,Bobo_Dioulasso,1961,1,1,1,1,3,False
34416,Bobo_Dioulasso,1961,1,1,1,1,4,False
45687,Bobo_Dioulasso,1961,1,1,1,1,5,False


In [7]:
saison_df[saison_df['Saison_Pluie']==False]

,Station,Year,Trimestre,Semestre,Month,Decade,Day,Saison_Pluie
603,Bobo_Dioulasso,1961,1,1,1,1,1,False
11874,Bobo_Dioulasso,1961,1,1,1,1,2,False
23145,Bobo_Dioulasso,1961,1,1,1,1,3,False
34416,Bobo_Dioulasso,1961,1,1,1,1,4,False
45687,Bobo_Dioulasso,1961,1,1,1,1,5,False
...,...,...,...,...,...,...,...,...
304304,Po,2021,4,2,12,3,27,False
315575,Po,2021,4,2,12,3,28,False
326147,Po,2021,4,2,12,3,29,False
336486,Po,2021,4,2,12,3,30,False


In [8]:
saison_df.to_csv('saison.csv')